In [1]:
import numpy as np
import os
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from natsort import natsorted
import string
import operator
import math


In [2]:
def loading_file(file):
	with open(file, 'r', errors ="surrogateescape") as f:
		ele = f.read()
	f.close()
	return ele

def rem_(strr):
    strr = strr.replace(".","")
    strr = strr.replace("-","")
    strr = strr.replace("'","")
    strr = strr.replace('"','')
    return strr

def Processing(input_str):
	tokenizer = TweetTokenizer()
	lst_token = tokenizer.tokenize(input_str.lower()) # Did Tokenization and Changed to lowercase here
	return lst_token


In [3]:
root_path = os.getcwd()
files = natsorted(os.listdir(root_path  + "\\Poems"))

In [4]:
Tokenized_Word_list = []
for file in files:
    ele = loading_file(root_path  + "\\Poems"+ "//" + file)
    ele = rem_(ele)
    res=Processing(ele)
    Tokenized_Word_list.append(res)

In [5]:
len(Tokenized_Word_list)

1860

In [6]:
class BM25:

    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus):

        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

    def _score(self, query, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

In [7]:
def input_process(str):
    str = rem_(str)
    res = Processing(str)
    return res
Phrase_Query = input("Enter your Phrase Query: ")
query= input_process(Phrase_Query)
bm25 = BM25()
bm25.fit(Tokenized_Word_list)
scores = bm25.search(query)

for score, doc in zip(scores, files):
    score = round(score, 3)
    print(str(score) + '\t' + doc)

5.048	aa gaii yaad shaam dhalte hii m_muneer niyazi.txt
1.073	aa hii jaaegii sahar matlaa e imkaan to khulaa majroo_majrooh sultanpuri.txt
2.09	aa jaaen ham nazar jo koii dam bahut hai yaan _meer taqi meer.txt
4.827	aa nikal ke maidaan men do rukhii ke khaane se majroo_majrooh sultanpuri.txt
5.87	aab o havaa hai barsar e paikaar kaun hai s_saleem kausar.txt
9.522	aabla paa koii guzraa thaa jo pichhle san men majroo_majrooh sultanpuri.txt
6.585	aadhon kii taraf se kabhii paunon kii taraf se _adil mansuri.txt
1.295	aadmii aadmii se miltaa hai jiga_jigar moradabadi.txt
1.304	aadmii na itnaa bhii duur ho zamaane se shakee_shakeel badayuni 3.txt
5.472	aadmii vaqt par gayaa hoga_jaun eliya.txt
9.541	aae hain miir kaafir ho kar khudaa ke ghar men _meer taqi meer.txt
1.607	aae ham gaalib o iqbaal ke nagmaat ke baad ali _ali sardar jafri.txt
0.203	aae kuchh abr kuchh sharaab aae fai_faiz ahmad faiz.txt
3.674	aag ashk e garm ko lage jii kyaa hii jal gayaa momi_momin khan momin.txt
1.577	aag kaa 

In [11]:
bm25scores = dict(zip(files, scores))
sorted_d = dict( sorted(bm25scores.items(), key=operator.itemgetter(1),reverse=True))
list(sorted_d.items())[:5]

[('apne chehre se jo zaahir hai chhupaaen kaise wa_waseem barelvi.txt',
  75.22713524444431),
 ('ab jo koii puuchhe bhii to us se kyaa sharh e haalaat karen fai_faiz ahmad faiz.txt',
  16.6437642482313),
 ('vo mujh ko kyaa bataanaa chaahtaa hai wa_waseem barelvi.txt',
  15.024245292489168),
 ('bulaatii hai magar jaane kaa naiin _rahat indori.txt', 14.592487563330831),
 ('jo dil pe guzartii hai vo samjhaa nahiin sakte shakee_shakeel badayuni 3.txt',
  14.034459484790927)]